# 라이브러리 및 데이터 다운로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import nltk
import string
import tensorflow_hub as hub
import tensorflow as tf
import sklearn

In [ ]:
data = pd.read_csv("../input/nlp-getting-started/train.csv")
X_test = pd.read_csv("../input/nlp-getting-started/test.csv")
submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
y = data["target"]

In [ ]:
data

In [ ]:
X = data["text"]

In [ ]:
X_for_tree = data[["keyword", "location"]]

텍스트 데이터를 이용한 LSTM과 Tabular data를 이용하여 RandomForest를 학습시키기 위하여 데이터를 분할해주었다.  
__X__ 는 LSTM을 위한 텍스트 데이터이며 __X_for_tree__ 는 RandomForest를 위한 Tabular data이다.

# 노이즈 제거
텍스트 데이터는 많은 노이즈를 가지고 있으므로 제거해주는 과정은 필수이다.

In [ ]:
X[:10]

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
X = pd.Series(map(clean_text, X))

In [ ]:
X[:10]

# 토큰화  
모델이 문장을 한번에 학습하는게 아닌 단어 별로 학습할 수 있도록 토큰화 시켜준다.

In [ ]:
X = [nltk.word_tokenize(sentence) for sentence in X]

In [ ]:
pd.DataFrame(X[:5])

# 불용어 제거   
텍스트 데이터에는 모델 학습에 큰 의미를 주지 않는 불용어가 다량 존재한다.  
[I see fire at the mountain] -> [see fire mountain] 이와 같은 식으로 전처리를 진행 한다.  
상당히 어려워보이는 작업이지만 nltk 라이브러리에는 이미 영어 불용어 사전이 구현되어 있기 때문에 이를 사용하여 간단히 가능하다.

In [ ]:
nltk.corpus.stopwords.words('english')[:5]

In [ ]:
pd.DataFrame(X[:5])

In [ ]:
def remove_stopwords(text):
    words = [w for w in text if w not in nltk.corpus.stopwords.words('english')]
    return words

X = list(map(remove_stopwords, X))

In [ ]:
pd.DataFrame(X[:5])

# 정수 인코딩
모델은 단어 그 자체를 학습하지 못하기 때문에 정수로 인코딩 해주어야 한다.

In [ ]:
pd.DataFrame(X[:5])

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

In [ ]:
pd.DataFrame(X[:5])

# 패딩
LSTM을 이용하여 텍스트 데이터를 훈련할 예정인데 RNN 계열 모델은 시쿼스가 너무 길어지면 학습이 잘 진행되지 않는다.  
데이터를 그래도 사용할 시 가장 긴 길이를 기준으로 패딩하게 되는데 이를 방지하지 위하여 정보를 너무 잃지 않는 길이로 패딩하였다.

In [ ]:
print("Average langth of tweet:", sum(map(len, X))/len(X))
plt.hist([len(tweet) for tweet in X])

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
ax1.hist([len(X[i]) for i in range(len(X)) if y[i]==1])
ax1.set_title("Real")
ax2.hist([len(X[i]) for i in range(len(X)) if y[i]==0], color="orange")
ax2.set_title("Fake")

In [ ]:
count=0
for tweet in X:
    if len(tweet) > 20:
        count+=1
print((len(X)-count) / len(X))

In [ ]:
X = keras.preprocessing.sequence.pad_sequences(X, maxlen=20, padding="post")

아래는 Test data를 위하여 생성한 위의 과정을 모두 합치 함수이다.

In [ ]:
def preprocess(text_sequence):
    text_sequence = pd.Series(map(clean_text, text_sequence))    
    text_sequence = [nltk.word_tokenize(sentence) for sentence in text_sequence]
    text_sequence = list(map(remove_stopwords, text_sequence))    
    text_sequence =  tokenizer.texts_to_sequences(text_sequence)
    text_sequence = keras.preprocessing.sequence.pad_sequences(text_sequence, maxlen=20, padding="post") 
    return text_sequence

# 테이블형 데이터 인코딩 및 데이터 split
테이블형 데이터에서 __keyword, location__  컬럼 역시 정수로 만들어주어야 한다.  
sklearn의 LabelEncoder를 이용하여 전처리 하였으며 보이지 않은 데이터에 대하여서 nan으로 만들어주었다.  
이미 인코더가 nan을 학습했으므로 문제 없이 동작한다.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, random_state=42)
X_train_tree, X_valid_tree, y_train, y_valid = train_test_split(X_for_tree, y, stratify=y, random_state=42)

In [ ]:
k_encoder = LabelEncoder()
X_train_tree["keyword"] = k_encoder.fit_transform(X_train_tree["keyword"])
X_valid_tree["keyword"] = k_encoder.transform(X_valid_tree["keyword"])

l_encoder = LabelEncoder()
X_train_tree["location"] = l_encoder.fit_transform(X_train_tree["location"])

In [ ]:
temp = np.array(X_valid_tree["location"])

for index in range(len(temp)):
    if temp[index] not in l_encoder.classes_:
        temp[index] = np.nan
        
X_valid_tree["location"] = temp
X_valid_tree["location"] = l_encoder.transform(X_valid_tree["location"])

# 모델 훈련

In [ ]:
def make_model(iter = 1, hidden=100):
    Input = keras.Input(shape=[20])

    x = keras.layers.Reshape((20, 1))(Input)
    
    for _ in range(iter):
        x = keras.layers.Bidirectional(keras.layers.LSTM(hidden, return_sequences=True))(x)
        
    x = keras.layers.Bidirectional(keras.layers.LSTM(hidden))(x)
    x = keras.layers.Dense((hidden+40)/2, activation="relu")(x)
    output = keras.layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=Input, outputs=output)
    model.compile(loss="binary_crossentropy", metrics="accuracy", optimizer="adam")
    return model

In [ ]:
model = make_model(1, 100)
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=30)

I grid search model shape by iter and hidden and I use parameteres above to train text data.

In [ ]:
plt.figure(figsize=(14, 10))
plt.plot(history.history["accuracy"], label="Train")
plt.plot(history.history["val_accuracy"], label="Valid")
plt.legend()

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
tree_model = RandomForestClassifier()
tree_model.fit(X_train_tree, y_train)

In [ ]:
X_valid_tree

In [ ]:
tree_model.score(X_valid_tree, y_valid)

Tree모델이 LSTM보다 더 좋은 성능을 보인다.  
자연어 전처리에 대한 지식과 RNN계열 지식이 부족하여 복잡한 데이터를 제대로 학습하는 모델을 만들지 못한것 같다.

In [ ]:
tree_score = tree_model.predict_proba(X_valid_tree)

In [ ]:
ann_score = np.concatenate((1-model.predict(X_valid), model.predict(X_valid)), axis=1)

In [ ]:
tree_score

In [ ]:
ann_score

In [ ]:
sklearn.metrics.accuracy_score(np.argmax(tree_score + ann_score, axis=1), y_valid)

각각의 라벨값에 대한 확신 정도?를 합쳐서 가장 높을 값을 선택하는 방식으로 Soft Voting Ensemble을 진행하였다.

# 제출  
제출 전에 Train 데이터를 모두 사용하여 모델을 재학습 시킬 필요가 있다.

In [ ]:
model = make_model(1, 100)
model.fit(X,y, epochs=30)

In [ ]:
k_encoder = LabelEncoder()
data["keyword"] = k_encoder.fit_transform(data["keyword"])
l_encoder = LabelEncoder()
data["location"] = l_encoder.fit_transform(data["location"])

In [ ]:
tree_model = RandomForestClassifier()
tree_model.fit(data[["keyword", "location"]], y)

X_test에 대하여서 똑같이 전처리 해주어야 한다.

In [ ]:
X_test_text = preprocess(X_test["text"])

In [ ]:
temp = np.array(X_test["location"])

for index in range(len(temp)):
    if temp[index] not in l_encoder.classes_:
        temp[index] = np.nan
        
X_test["location"] = temp

In [ ]:
X_test["keyword"] = k_encoder.transform(X_test["keyword"])
X_test["location"] = l_encoder.transform(X_test["location"])

In [ ]:
ann_score = np.concatenate((1-model.predict(X_test_text), model.predict(X_test_text)), axis=1)

In [ ]:
tree_score = tree_model.predict_proba(X_test[["keyword", "location"]])

In [ ]:
Final_score = np.argmax(tree_score+ann_score, axis=1)

In [ ]:
submission["target"] = Final_score

In [ ]:
submission.to_csv("submission.csv", index=None)